In [1]:
import osmium
import pygeos
import geopandas as gpd
from shapely.geometry import Point

# Étape 1 : Construire un DataFrame GeoPandas pour tous les POIs
class POIHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.data = []

    def node(self, n):
        if "amenity" in n.tags:
            self.data.append({"geometry": Point(n.location.lon, n.location.lat), "tags": dict(n.tags)})

def load_pois_to_geodataframe(pbf_file):
    handler = POIHandler()
    handler.apply_file(pbf_file)
    return gpd.GeoDataFrame(handler.data, geometry="geometry", crs="EPSG:4326")

pois_gdf = load_pois_to_geodataframe("data_pop_density/france-latest.osm.pbf")

# Étape 2 : Convertir en index spatial
pois_gdf = pois_gdf.to_crs("EPSG:3857")  # Projeter en mètres
poi_index = pygeos.STRtree(pygeos.from_shapely(pois_gdf.geometry))

# Étape 3 : Interroger les POIs proches avec des buffers
def fetch_pois_nearby(lat, lon, radius=500):
    point = pygeos.points(lon, lat)
    buffer = pygeos.buffer(point, radius)
    matches = poi_index.query(buffer)
    return pois_gdf.iloc[matches]

# Exemple pour un point spécifique
pois_nearby = fetch_pois_nearby(48.8566, 2.3522, radius=500)
print(pois_nearby)


/home/phdel/.pyenv/versions/3.10.6/envs/upfund_dvf/lib/python3.10/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.11.4-CAPI-1.17.4) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


Empty GeoDataFrame
Columns: [geometry, tags]
Index: []
